历史遗留问题：不能接受不同大小混合。
https://chatgpt.com/share/685be7ec-bf08-8007-8ee7-24555472c802
全部都在解决这个问题，提出更多新思路。

In [1]:

import numpy as np
import random

def mix_npy_data(npy_file_list, output_path, samples_per_file=(1, 3), seed=None):
    """
    从多个 .npy 文件中轮换抽取 1–3 行，直至所有文件的行都被使用完毕，生成混合数据集。

    参数：
    - npy_file_list: List[str]，所有要读取的 .npy 文件路径
    - output_path: str，保存合并输出的 .npy 文件路径
    - samples_per_file: Tuple[int, int]，每次从某文件中随机抽取的行数范围（闭区间）
    - seed: int，可选，设定随机种子以保证可复现性

    返回：
    - output_path: 保存成功后的路径
    """
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    # 加载每个文件的数据和初始化未抽取行索引
    file_data = []  # [(data_array, remaining_indices_list), ...]
    for file in npy_file_list:
        arr = np.load(file)  # shape: (N, L)
        indices = list(np.random.permutation(len(arr)))  # 打乱行索引
        file_data.append((arr, indices))

    all_samples = []
    file_count = len(file_data)
    finished = [False] * file_count  # 每个文件是否处理完毕的标志
    active_files = file_count  # 当前仍有剩余行的文件数
    pointer = 0  # 当前轮到哪个文件抽取

    while active_files > 0:
        arr, indices = file_data[pointer]

        if not finished[pointer]:
            # 当前文件仍有剩余行
            k = random.randint(*samples_per_file)
            k = min(k, len(indices))  # 如果不足 k 行，取剩下所有行
            chosen = indices[:k]
            indices[:] = indices[k:]  # 从剩余列表中移除已使用的行
            file_data[pointer] = (arr, indices)  # 更新对应文件的剩余数据

            # 加入最终混合数据中
            all_samples.append(arr[chosen])

            # 如果该文件已抽完，标记为完成
            if len(indices) == 0:
                finished[pointer] = True
                active_files -= 1

        # 移动到下一个文件（循环轮换）
        pointer = (pointer + 1) % file_count

    # 合并所有批次并保存
    mixed_array = np.concatenate(all_samples, axis=0)
    np.save(output_path, mixed_array)
    print(f"数据已保存到 {output_path}，总行数: {mixed_array.shape[0]}")
    return output_path



In [2]:
npy_file_list = [
    "/Users/guwenlan/Desktop/DURF/Gernerated_/input_for_2_qubits_mixed_10000_datapoints.npy",
    '/Users/guwenlan/Desktop/DURF/Gernerated_/input_for_3_qubits_mixed_10000_datapoints.npy',
    '/Users/guwenlan/Desktop/DURF/Gernerated_/input_for_4_qubits_mixed_10000_datapoints.npy'
]
output_path = '/Users/guwenlan/Desktop/DURF/Mixed_data_2_3_4.npy'
mix_npy_data(npy_file_list, output_path, samples_per_file=(1, 3), seed=None)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 16 and the array at index 1 has size 64